In [1]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [2]:
data = pd.read_csv("data/cse_final.csv")
data.head()

,Unnamed: 0,name,title,email,lab,interests,website,office,phone,clarity,preparedness,respect,research,students taught,terms taught
0,0,"Bansal, Nikhil",Patrick C. Fischer Professor of Theoretical Co...,bansaln@umich.edu,['theory'],"['Algorithms', 'Optimization', 'Machine Learni...",https://bansal.engin.umich.edu/,3637 Beyster Bldg.,NaN,100.0,100.0,100.0,False,1144.0,4.0
1,1,"Bodwin, Greg","Assistant Professor, Electrical Engineering & ...",bodwin@umich.edu,['theory'],"['Graph Algorithms', 'Data Structures', 'Sketc...",https://bodwin.engin.umich.edu/,3609 Beyster Bldg,(734)647-1902,NaN,NaN,NaN,False,776.0,6.0
2,2,"Cheraghchi, Mahdi","Associate Professor, Electrical Engineering an...",mahdich@umich.edu,['theory'],"['Information And Coding Theory', 'Cryptograph...",https://mahdi.ch/,3603 Beyster Bldg.,(734) 763-9165,NaN,NaN,NaN,True,434.0,8.0
3,3,"Compton, Kevin J.","Associate Professor Emeritus, Electrical Engin...",kjc@umich.edu,"['theory', 'systems']","['Theory Of Computation', 'Complexity Of Combi...",http://web.eecs.umich.edu/~kjc,3603 Beyster Bldg.,(734) 763-9165,97.0,99.0,99.0,False,4118.0,57.0
4,4,"Dereziński, Michał","Assistant Professor, Electrical Engineering an...",derezin@umich.edu,"['theory', 'ai']",['Machine Learning And Statistical Learning Th...,https://web.eecs.umich.edu/~derezin/,2649 Beyster,NaN,NaN,NaN,NaN,False,1245.0,5.0


In [ ]:
# change research to bool
data['research'] = data['research'].astype(bool)

In [3]:
emails = data["email"]
uniqnames = []
for email in emails:
    uniqnames.append(email.replace("@umich.edu",""))

In [4]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # this keeps the window open post execution/errors/ changing frames (explained later)

prefs = {"download.default_directory": "C:\\Users\\eaalc\\Downloads",  # note need to change this
         "directory_upgrade": True}
chrome_options.add_experimental_option("prefs", prefs)

In [5]:
# open semi irrelevant website to login
driver = webdriver.Chrome('/opt/homebrew/bin/chromedriver', options=chrome_options)
driver.get("https://atlas.ai.umich.edu/my-dashboard/")

/var/folders/zv/431d3wns2pd4pglh7zpflby00000gn/T/ipykernel_42064/583303880.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/opt/homebrew/bin/chromedriver', options=chrome_options)


In [6]:
def webscrape_stats():
    elts = driver.find_elements(By.CLASS_NAME, 'eval-section-stat')
    return [elt.text for elt in elts]

In [7]:
students_taught = []
terms_taught = []
def webscrape_teaching():
    elem = driver.find_elements(By.XPATH, "//p[@class = 'text-xlarge bold blue-highlight-text']")
    return elem[0].text, elem[1].text
#     print(students_taught)
#     print(terms_taught)

In [ ]:
for uniqname in uniqnames: 
    driver.get("https://atlas.ai.umich.edu/instructor/" + uniqname + "/")
    print(uniqname)
    try: 
        element = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'eval-section-stat'))
            )
        webscrape_stats()
#         print(nums)
        t_elem = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class = 'text-xlarge bold blue-highlight-text']"))
            )
        webscrape_teaching()
        print("entered try")
    except:
        print("stats unavailable")
        t_elem = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class = 'text-xlarge bold blue-highlight-text']"))
            )
        webscrape_teaching()
        
        
    finally:
        # how to move on to next prof?
        for i in range(3):
            nums.append(0)
            #students_taught.append(0)
            #terms_taught.append(0)
        continue

In [8]:
for ind, row in data.iterrows(): 
    uniqname = row['email'].replace("@umich.edu","")
    driver.get("https://atlas.ai.umich.edu/instructor/" + uniqname + "/")
    print(uniqname)
    try: 
        element = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'eval-section-stat'))
            )
        stats = webscrape_stats()
        data.at[ind, 'clarity'] = stats[0]
        data.at[ind, 'preparedness'] = stats[1]
        data.at[ind, 'respect'] = stats[2]
        
    except Exception as e:
        print('stats unavailable')
        data.at[ind, 'clarity'] = np.nan
        data.at[ind, 'preparedness'] = np.nan
        data.at[ind, 'respect'] = np.nan
#         print(nums)
    try:
        t_elem = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class = 'text-xlarge bold blue-highlight-text']"))
            )
        teach_stats = webscrape_teaching()
        data.at[ind, 'students taught'] = teach_stats[0]
        data.at[ind, 'terms taught'] = teach_stats[1]
        
    except Exception as e:
        print("teaching stats unavailable")
        data.at[ind, 'students taught'] = np.nan
        data.at[ind, 'terms taught'] = np.nan
        

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs
bansaln
bodwin
stats unavailable
mahdich
kjc
stats unavailable
derezin
benfish
gurevich
stats unavailable
teaching stats unavailable
vvh
stats unavailable
euiwoong
cpeikert
pettie
rounds
stats unavailable
teaching stats unavailable
thsa
qstout
nbanovic
baveja
lburdick
chaijy
dcsmc
stats unavailable
jjcorso
fouhey
anhong
profdj
stats unavailable
justincj
dkoutra
kuipers
skutty
honglak
mmakar
stats unavailable
mihalcea
emilykmp
ebolson
stats unavailable
teaching stats unavailable
vrncapr
stats unavailable
teaching stats unavailable
wangluxy
stats unavailable
xwanghci
wellman
stats unavailable
teaching stats unavailable
wiensj
stellayu
stats unavailable
austin
valeria
stats unavailable
teaching stats unavailable
pmchen
teaching stats unavailable
conway
stats unavailable
teaching stats unavailable
reetudas
davidson
stats unavailable
teaching stats unavailable
rdreslin
jhayes
ryanph
stats unavailable
teaching stats unavaila

In [9]:
data

,Unnamed: 0,name,title,email,lab,interests,website,office,phone,clarity,preparedness,respect,research,students taught,terms taught
0,0,"Bansal, Nikhil",Patrick C. Fischer Professor of Theoretical Co...,bansaln@umich.edu,['theory'],"['Algorithms', 'Optimization', 'Machine Learni...",https://bansal.engin.umich.edu/,3637 Beyster Bldg.,NaN,100%,100%,100%,False,"1,144",4
1,1,"Bodwin, Greg","Assistant Professor, Electrical Engineering & ...",bodwin@umich.edu,['theory'],"['Graph Algorithms', 'Data Structures', 'Sketc...",https://bodwin.engin.umich.edu/,3609 Beyster Bldg,(734)647-1902,NaN,NaN,NaN,False,776,6
2,2,"Cheraghchi, Mahdi","Associate Professor, Electrical Engineering an...",mahdich@umich.edu,['theory'],"['Information And Coding Theory', 'Cryptograph...",https://mahdi.ch/,3603 Beyster Bldg.,(734) 763-9165,99%,97%,99%,True,434,8
3,3,"Compton, Kevin J.","Associate Professor Emeritus, Electrical Engin...",kjc@umich.edu,"['theory', 'systems']","['Theory Of Computation', 'Complexity Of Combi...",http://web.eecs.umich.edu/~kjc,3603 Beyster Bldg.,(734) 763-9165,NaN,NaN,NaN,False,"4,118",57
4,4,"Dereziński, Michał","Assistant Professor, Electrical Engineering an...",derezin@umich.edu,"['theory', 'ai']",['Machine Learning And Statistical Learning Th...,https://web.eecs.umich.edu/~derezin/,2649 Beyster,NaN,63%,48%,91%,False,"1,245",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,"Paoletti, David Robert","Lecturer IV, Electrical Engineering and Comput...",paoletti@umich.edu,['cse-teaching'],NaN,NaN,2645 Beyster,734-763-8205,97%,96%,96%,False,NaN,NaN
107,107,"Razak, Saquib","LEO Lecturer III, Electrical Engineering & Com...",razak@umich.edu,['cse-teaching'],NaN,NaN,2636 Beyster,NaN,NaN,NaN,NaN,False,"1,410",2
108,108,"Ringenberg, Jeffrey S.","Lecturer IV, Electrical Engineering and Comput...",jringenb@umich.edu,['cse-teaching'],"['Mobile Learning Software Development', 'Tact...",NaN,2633 Beyster Bldg.,(734) 763-2751,93%,93%,95%,False,NaN,NaN
109,109,"Saleem, Sofia","Lecturer I, Electrical Engineering & Computer ...",sofias@umich.edu,['cse-teaching'],NaN,NaN,3917 Beyster Bldg.,NaN,89%,78%,91%,False,"5,678",8


In [ ]:
stats = [nums[x:x+3] for x in range(0, len(nums), 3)]

for ind, x in enumerate(stats):
    for i, y in enumerate(x):
        if type(y) == str:
            stats[ind][i] = int(y[:-1])
stats

In [18]:
data['preparedness'] = data['preparedness'].apply(lambda x: int(x[:-1]) if type(x)==str else np.nan)
data['clarity'] = data['clarity'].apply(lambda x: int(x[:-1]) if type(x)==str else np.nan)
data['respect'] = data['respect'].apply(lambda x: int(x[:-1]) if type(x)==str else np.nan)

data.head()

,Unnamed: 0,name,title,email,lab,interests,website,office,phone,clarity,preparedness,respect,research,students taught,terms taught
0,0,"Bansal, Nikhil",Patrick C. Fischer Professor of Theoretical Co...,bansaln@umich.edu,['theory'],"['Algorithms', 'Optimization', 'Machine Learni...",https://bansal.engin.umich.edu/,3637 Beyster Bldg.,NaN,100.0,100.0,100.0,False,"1,144",4
1,1,"Bodwin, Greg","Assistant Professor, Electrical Engineering & ...",bodwin@umich.edu,['theory'],"['Graph Algorithms', 'Data Structures', 'Sketc...",https://bodwin.engin.umich.edu/,3609 Beyster Bldg,(734)647-1902,NaN,NaN,NaN,False,776,6
2,2,"Cheraghchi, Mahdi","Associate Professor, Electrical Engineering an...",mahdich@umich.edu,['theory'],"['Information And Coding Theory', 'Cryptograph...",https://mahdi.ch/,3603 Beyster Bldg.,(734) 763-9165,99.0,97.0,99.0,True,434,8
3,3,"Compton, Kevin J.","Associate Professor Emeritus, Electrical Engin...",kjc@umich.edu,"['theory', 'systems']","['Theory Of Computation', 'Complexity Of Combi...",http://web.eecs.umich.edu/~kjc,3603 Beyster Bldg.,(734) 763-9165,NaN,NaN,NaN,False,"4,118",57
4,4,"Dereziński, Michał","Assistant Professor, Electrical Engineering an...",derezin@umich.edu,"['theory', 'ai']",['Machine Learning And Statistical Learning Th...,https://web.eecs.umich.edu/~derezin/,2649 Beyster,NaN,63.0,48.0,91.0,False,"1,245",5


In [31]:
data['students taught'] = data['students taught'].apply(lambda x: x.replace(',', '') if type(x) == str else x).astype(np.float64)
data['terms taught'] = data['terms taught'].apply(lambda x: x.replace(',', '') if type(x) == str else x).astype(np.float64)

data.head()

,Unnamed: 0,name,title,email,lab,interests,website,office,phone,clarity,preparedness,respect,research,students taught,terms taught
0,0,"Bansal, Nikhil",Patrick C. Fischer Professor of Theoretical Co...,bansaln@umich.edu,['theory'],"['Algorithms', 'Optimization', 'Machine Learni...",https://bansal.engin.umich.edu/,3637 Beyster Bldg.,NaN,100.0,100.0,100.0,False,1144.0,4.0
1,1,"Bodwin, Greg","Assistant Professor, Electrical Engineering & ...",bodwin@umich.edu,['theory'],"['Graph Algorithms', 'Data Structures', 'Sketc...",https://bodwin.engin.umich.edu/,3609 Beyster Bldg,(734)647-1902,NaN,NaN,NaN,False,776.0,6.0
2,2,"Cheraghchi, Mahdi","Associate Professor, Electrical Engineering an...",mahdich@umich.edu,['theory'],"['Information And Coding Theory', 'Cryptograph...",https://mahdi.ch/,3603 Beyster Bldg.,(734) 763-9165,99.0,97.0,99.0,True,434.0,8.0
3,3,"Compton, Kevin J.","Associate Professor Emeritus, Electrical Engin...",kjc@umich.edu,"['theory', 'systems']","['Theory Of Computation', 'Complexity Of Combi...",http://web.eecs.umich.edu/~kjc,3603 Beyster Bldg.,(734) 763-9165,NaN,NaN,NaN,False,4118.0,57.0
4,4,"Dereziński, Michał","Assistant Professor, Electrical Engineering an...",derezin@umich.edu,"['theory', 'ai']",['Machine Learning And Statistical Learning Th...,https://web.eecs.umich.edu/~derezin/,2649 Beyster,NaN,63.0,48.0,91.0,False,1245.0,5.0


In [32]:
data

,Unnamed: 0,name,title,email,lab,interests,website,office,phone,clarity,preparedness,respect,research,students taught,terms taught
0,0,"Bansal, Nikhil",Patrick C. Fischer Professor of Theoretical Co...,bansaln@umich.edu,['theory'],"['Algorithms', 'Optimization', 'Machine Learni...",https://bansal.engin.umich.edu/,3637 Beyster Bldg.,NaN,100.0,100.0,100.0,False,1144.0,4.0
1,1,"Bodwin, Greg","Assistant Professor, Electrical Engineering & ...",bodwin@umich.edu,['theory'],"['Graph Algorithms', 'Data Structures', 'Sketc...",https://bodwin.engin.umich.edu/,3609 Beyster Bldg,(734)647-1902,NaN,NaN,NaN,False,776.0,6.0
2,2,"Cheraghchi, Mahdi","Associate Professor, Electrical Engineering an...",mahdich@umich.edu,['theory'],"['Information And Coding Theory', 'Cryptograph...",https://mahdi.ch/,3603 Beyster Bldg.,(734) 763-9165,99.0,97.0,99.0,True,434.0,8.0
3,3,"Compton, Kevin J.","Associate Professor Emeritus, Electrical Engin...",kjc@umich.edu,"['theory', 'systems']","['Theory Of Computation', 'Complexity Of Combi...",http://web.eecs.umich.edu/~kjc,3603 Beyster Bldg.,(734) 763-9165,NaN,NaN,NaN,False,4118.0,57.0
4,4,"Dereziński, Michał","Assistant Professor, Electrical Engineering an...",derezin@umich.edu,"['theory', 'ai']",['Machine Learning And Statistical Learning Th...,https://web.eecs.umich.edu/~derezin/,2649 Beyster,NaN,63.0,48.0,91.0,False,1245.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,"Paoletti, David Robert","Lecturer IV, Electrical Engineering and Comput...",paoletti@umich.edu,['cse-teaching'],NaN,NaN,2645 Beyster,734-763-8205,97.0,96.0,96.0,False,NaN,NaN
107,107,"Razak, Saquib","LEO Lecturer III, Electrical Engineering & Com...",razak@umich.edu,['cse-teaching'],NaN,NaN,2636 Beyster,NaN,NaN,NaN,NaN,False,1410.0,2.0
108,108,"Ringenberg, Jeffrey S.","Lecturer IV, Electrical Engineering and Comput...",jringenb@umich.edu,['cse-teaching'],"['Mobile Learning Software Development', 'Tact...",NaN,2633 Beyster Bldg.,(734) 763-2751,93.0,93.0,95.0,False,NaN,NaN
109,109,"Saleem, Sofia","Lecturer I, Electrical Engineering & Computer ...",sofias@umich.edu,['cse-teaching'],NaN,NaN,3917 Beyster Bldg.,NaN,89.0,78.0,91.0,False,5678.0,8.0


In [ ]:
data.replace(0, np.nan, inplace=True)
data.head()

In [34]:
data.to_csv('data/cse_final.csv', index=False)